# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


In [5]:
cold_users = list(set(users_test) - set(common_users))

print(len(cold_users)) 

2365


 найдите топ-100 наиболее популярных книг (учитывая и их среднюю оценку) за последние несколько лет. Например, с 2015 года и со средней оценкой не меньше 4

In [6]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values("popularity_weighted", ascending=False) # ваш код здесь #

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.loc[item_popularity["avg_rating"] >= 4].iloc[:100, ] # ваш код здесь # 

In [7]:
top_k_pop_items.shape

(100, 4)

In [8]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


In [9]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

In [10]:
cold_user_recs.shape[0] / cold_user_items_no_avg_rating_idx.shape[0]

0.19768403639371382

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae =  mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [12]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 


Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

Разряженность матрицы

In [13]:
1 - events[['user_id', 'item_id', 'rating']].shape[0] / (len(events['user_id'].unique()) * len(events['item_id'].unique()))

0.9993451160571009

In [14]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

In [15]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [16]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae) 

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [17]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set) 

In [18]:
mae = accuracy.mae(random_predictions)

MAE:  1.0018


In [19]:
(1.0018 - 0.64528 ) / (0.64528   / 100)

55.2504339201587

Удалим редкие айтемы(те, которык читали меньше 2 пульзователей) и повторим наш рассчёт

In [20]:
count_it_read = events.groupby('item_id')['user_id'].agg('count')
count_it_read = count_it_read[count_it_read > 3]

In [21]:
filtred_events = events[events['item_id'].isin(count_it_read)]

In [22]:
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(filtred_events[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

svd_filtred_model = SVD(n_factors=100, random_state=0)

svd_filtred_model.fit(surprise_train_set)

Сделаем прогноз.

In [23]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_filtred_model.test(surprise_test_set) 

In [24]:
rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae) 

RMSE: 0.9174
MAE:  0.6944
0.9173576953967969 0.6944102521146956


In [25]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[(events['user_id']==user_id) & (events['is_read']==True) ]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"]) 

In [26]:
get_recommendations_svd(1296647, items, events_test, svd_model) 

,item_id,score
0,7864312,4.981188
1,25793186,4.912001
2,12174312,4.898052
3,13208,4.894869
4,33353628,4.891661


In [27]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1169023
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,2014-06-04,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,2014-05-29,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,2014-05-27,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,2014-05-22,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,2014-06-02,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,2014-05-22,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,2014-05-08,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,2014-05-17,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,2014-05-10,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,2014-05-04,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."


In [28]:
new_row = {'user_id': -1, 
           'item_id': 28143415, 
           'started_at': '2015-07-12',
           'read_at': '2015-07-17', 
           'is_read': True, 
           'rating': 4, 
           'is_reviewed': False, 
           'started_at_month': '2015-07-01', }

events_train.loc[events_train.index.max() + 1] = new_row

new_row = {'user_id': -1, 
           'item_id': 3, 
           'started_at': '2015-08-12',
           'read_at': '2015-08-17', 
           'is_read': True, 
           'rating': 4, 
           'is_reviewed': False, 
           'started_at_month': '2015-08-01', }

events_train.loc[events_train.index.max() + 1] = new_row

new_row = {'user_id': -1, 
           'item_id': 7757584, 
           'started_at': '2015-09-12',
           'read_at': '2015-09-17', 
           'is_read': True, 
           'rating': 5, 
           'is_reviewed': False, 
           'started_at_month': '2015-09-01', }

events_train.loc[events_train.index.max() + 1] = new_row

new_row = {'user_id': -1, 
           'item_id': 127566, 
           'started_at': '2015-09-12',
           'read_at': '2015-09-17', 
           'is_read': True, 
           'rating': 5, 
           'is_reviewed': False, 
           'started_at_month': '2015-09-01', }

events_train.loc[events_train.index.max() + 1] = new_row

/tmp/ipykernel_1907/128788422.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train.loc[events_train.index.max() + 1] = new_row
/tmp/ipykernel_1907/128788422.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train.loc[events_train.index.max() + 1] = new_row
/tmp/ipykernel_1907/128788422.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train.loc[events_train.index.max() + 1] = new_row
/tmp/ipykernel_1907

In [29]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = -1

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: -1
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2015-07-12,2015-07-17,4,"{'Fantasy': 50009, 'Young Adult': 15428, 'Fict..."
1,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,2015-08-12,2015-08-17,4,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
2,"Anne McCaffrey, Todd McCaffrey","Dragon's Time (Pern, #23)",2015-09-12,2015-09-17,5,"{'Fantasy': 247, 'Science Fiction': 95, 'Fanta..."
3,Anne McCaffrey,"The Skies of Pern (Pern, #16)",2015-09-12,2015-09-17,5,"{'Fantasy': 820, 'Science Fiction': 345, 'Fant..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,24812,5.000000,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
1,11221285,4.914296,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,22037424,4.908423,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
3,33353628,4.872179,Pénélope Bagieu,"Culottées #2 (Culottées, #2)","{'Sequential Art-Bande DessinÃ©e': 108, 'Femin..."
4,54741,4.872000,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."


In [36]:
events_train = events_train.loc[events_train['user_id'] != -1] 

# === Базовые подходы: коллаборативная фильтрация

In [39]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])


/tmp/ipykernel_1907/80071505.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_1907/80071505.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_1907/80071505.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [40]:
events_train["item_id_enc"].max()

43304

In [43]:
events["user_id"].unique().shape[0] * events["item_id"].unique().shape[0] / 1024 ** 3

16.71143686864525

In [44]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8) 

In [45]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3 

0.26370687410235405

In [47]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

/home/mle-user/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [49]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations 

In [54]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events["user_id"].unique()[np.random.randint(events["user_id"].unique().shape[0])]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=5)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1367611
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
14,"Italo Calvino, William Weaver",If on a Winter's Night a Traveler,2012-02-23,2012-02-27,4,"{'Fiction': 2430, 'Classics': 927, 'European L..."
15,"F. Scott Fitzgerald, Francis Scott Fitzgerald",The Great Gatsby,2012-02-12,2012-02-15,5,"{'Classics': 42010, 'Fiction': 19710, 'Academi..."
16,"Italo Calvino, Archibald Colquhoun",The Baron in the Trees,2012-02-11,2012-02-14,4,"{'Fiction': 671, 'Classics': 257, 'European Li..."
17,"Dai Sijie, Ina Rilke",Balzac and the Little Chinese Seamstress,2012-03-08,2012-03-09,4,"{'Fiction': 1407, 'Historical-Historical Ficti..."
18,"Natsume Sōseki, Edwin McClellan",Kokoro,2012-02-16,2012-02-17,4,"{'Fiction': 646, 'Cultural-Japan': 611, 'Asian..."
19,"Haruki Murakami, Philip Gabriel",Sputnik Sweetheart,2012-02-02,2012-02-03,5,"{'Fiction': 1783, 'Cultural-Japan': 719, 'Asia..."
20,Aldous Huxley,Brave New World,2012-03-09,2012-03-10,5,"{'Classics': 17717, 'Fiction': 12564, 'Science..."
21,"Aristophanes, Sarah Ruden",Lysistrata,2012-03-07,2012-03-08,5,"{'Plays': 1148, 'Classics': 1121, 'Drama': 335..."
22,Arthur C. Clarke,"3001: The Final Odyssey (Space Odyssey, #4)",2012-01-30,2012-01-31,4,"{'Science Fiction': 1076, 'Fiction': 387}"
23,"Honoré de Balzac, Ellen Marriage, George Saint...",Lost Illusions,2012-02-28,2012-03-05,5,"{'Classics': 369, 'Fiction': 252, 'Cultural-Fr..."


Рекомендации


,item_id_enc,score,item_id,author,title,genre_and_votes
0,493,0.553315,4671,"F. Scott Fitzgerald, Francis Scott Fitzgerald",The Great Gatsby,"{'Classics': 42010, 'Fiction': 19710, 'Academi..."
1,557,0.405924,5107,J.D. Salinger,The Catcher in the Rye,"{'Classics': 30431, 'Fiction': 17151, 'Young A..."
2,635,0.377276,5470,"George Orwell, Erich Fromm",1984,"{'Classics': 33441, 'Fiction': 21086, 'Science..."
3,533,0.338775,4981,Kurt Vonnegut Jr.,Slaughterhouse-Five,"{'Classics': 12568, 'Fiction': 11930, 'Science..."
4,561,0.336364,5129,Aldous Huxley,Brave New World,"{'Classics': 17717, 'Fiction': 12564, 'Science..."


In [55]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100) 

In [56]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"]) 

In [58]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet") 

In [59]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
) 

In [60]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg 

In [61]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5)) 

/tmp/ipykernel_1907/3905104878.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))


In [63]:
print(ndcg_at_5_scores.mean()) 

0.975946709792109


In [65]:
(ndcg_at_5_scores.shape[0] - ndcg_at_5_scores.isna().sum()) / ndcg_at_5_scores.shape[0]

0.35807624389737197

In [73]:
# HP & PhS
i2i_rec = als_model.similar_items(41745)

items[items['item_id_enc'].isin(i2i_rec[0])].head(5)

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
418822,34299826,Karen Gregory,Countless,'Is there anything that's concerning you?' Fel...,"{'Young Adult': 35, 'Contemporary': 28, 'Healt...",384,4.06,238,83,Bloomsbury Publishing Plc,2017,US,eng,Paperback,False,,9781408882504,"{'Academic': None, 'Academic-Academia': None, ...","Young Adult 35, Contemporary 28, Health-Mental...",42921
488701,2040488,Kenneth Bøgh Andersen,"Djævelens lærling (Den store djævlekrig, #1)",Nu er Kenneth Bogh Andersens prisbelonnede fan...,"{'Fantasy': 135, 'Young Adult': 35, 'European ...",310,4.09,1132,40,Host & Son,2007,US,dan,Paperback,False,8763806150,9788763806152,"{'Academic': None, 'Academic-Academia': None, ...","Fantasy 135, Young Adult 35, European Literatu...",19953
546146,28757261,"J.K. Rowling, Olly Moss",Harry Potter and the Philosopher's Stone,"""Turning the envelope over, his hand trembling...","{'Fantasy': 60042, 'Fiction': 17988, 'Young Ad...",<NA>,4.45,5212,127,,<NA>,US,eng,,True,,,"{'Academic': None, 'Academic-Academia': None, ...","Fantasy 60042, Fiction 17988, Young Adult 1793...",41745
679381,23344222,Amy Alward,"The Potion Diaries (Potion, #1)",When the Princess of Nova accidentally poisons...,"{'Fantasy': 271, 'Young Adult': 167, 'Romance'...",320,3.76,1506,278,Simon & Schuster UK,2015,US,eng,Paperback,False,1471143562,9781471143564,"{'Academic': None, 'Academic-Academia': None, ...","Fantasy 271, Young Adult 167, Romance 101, Fan...",39587
1088789,25255576,Louise O'Neill,Asking For It,It's the beginning of the summer in a small to...,"{'Young Adult': 552, 'Contemporary': 354, 'Fem...",346,4.05,6636,1213,Quercus UK,2015,US,eng,Paperback,False,,9781848664173,"{'Academic': None, 'Academic-Academia': None, ...","Young Adult 552, Contemporary 354, Feminism 29...",40520


# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков